**Server**

In [1]:
import socket, cv2, pickle,struct,imutils, threading

# Socket Create
server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)

host_name  = socket.gethostname()
host_ip = ""

port = 9999
socket_address = (host_ip,port)

# Socket Bind
server_socket.bind(socket_address)

# Socket Listen
server_socket.listen(5)
print("LISTENING AT:",socket_address)


        

LISTENING AT: ('', 9999)


**Streaming Video to Client**

In [2]:
def send(client_socket,addr):
        print('GOT CONNECTION FROM:',addr)
        if client_socket:
            vid = cv2.VideoCapture(0)

            while(vid.isOpened()):
                try:
                    img,frame = vid.read()
                    frame = imutils.resize(frame,width=320)
                    a = pickle.dumps(frame)
                    message = struct.pack("Q",len(a))+a
                    client_socket.sendall(message)
                except ConnectionAbortedError:
                    vid.release()
                    client_socket.close()
                    break

**Receiving Video from Client**

In [3]:
def recv():
    data = b""
    payload_size = struct.calcsize("Q")
    while True:
        while len(data) < payload_size:
            packet = client_socket.recv(4*1024) # 4K
            if not packet: break
            data+=packet
        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("Q",packed_msg_size)[0]

        while len(data) < msg_size:
            data += client_socket.recv(4*1024)
        frame_data = data[:msg_size]
        data  = data[msg_size:]
        frame = pickle.loads(frame_data)
        cv2.imshow("Client to server video",frame)
        if cv2.waitKey(1)  == 13:
            break
    cv2.destroyAllWindows()
    client_socket.close()
                    


In [ ]:
while True:
    client_socket,addr = server_socket.accept()
        
    if client_socket:
        t2=threading.Thread(target=recv)
        t2.start()
        t1=threading.Thread(target=send, args=(client_socket, addr))
        t1.start()
        


GOT CONNECTION FROM: ('192.168.43.195', 59584)


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\hp\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\hp\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-17d78f110390>", line 12, in send
    client_socket.sendall(message)
OSError: [WinError 10038] An operation was attempted on something that is not a socket

